# **Lab 3: approximations**
**Gustav Grevsten**

# **Abstract**

The purpose of this lab is to implement and test a method for approximating a real-valued function $f(x)$ using projections onto orthogonal vectors that span the $L^2$ function space. Specifically, we will approximate the function over a $1D$ mesh using piecewise linear functions derived from the projection of the function over the first two vectors in the set of Legendre polynomials. In the end, we conclude that the approximation yeilds accurate results that converge as the size of the mesh grows larger.

# **Set up environment**

In [46]:
# Load neccessary modules.
import numpy as np

# **Introduction**

For many problems of scientific computations, it is necessary to discretize the problem that is being worked on so that it can be handled by a computer. For this purpose, it might be necessary to create a piecewise linear approximation to the problem

Within the $L^2$ function space, it is possible to approximate functions that are members of this vector space by projecting them onto orthogonal basis vectors. A set of orthogonal basis vectors for this space are the Legendre polynomials, which can be used to find linear approximations of any square integrable functions. Using this fact, we will be able to construct an algorithm for finding piecewise linear approximation functions for arbitrary functions within this space.

# **Method**

The algorithm we will employ in this lab involves a projection of the function $f(x)$ onto the Legendre polynomials of degree 0 and 1, which are the vectors $P_0(x) = 1$ and $P_1(x) = x$. Since the Legendre polynomials are orthogonal and span the function space $L^2$, we can approximate any other function in the space by projecting it onto a subset of the set of Legendre polynomials using the inner product $(f(x), P_n(x)) = \int_{-1}^{1}P_n(x)f(x)dx$ to construct the projection $Proj(f(x), P_n(x)) = \frac{(f(x), P_n(x))}{||P_n(x)||}P_n(x)$. We can approximate this integral using the trapezoid rule. The sum of these projections thus approximate the function $f(x)$ in the range $[-1, 1]$.

(OBS: It should be noted that we wish to approximate functions in arbitrary intervalls $[a,b]$, where $a$ and $b$ are real values. In the implementation, we thus need to compensate for the fact that the Legendre polynomials are only orthogonal in $L^2(-1, 1)$ by performing a coordinate shift and calculating the integral $\int_{-1}^{1}P_n(x)f(\frac{(b-a)x}{2} - \frac{a+b}{2})dx$, thus giving us the approximate values $f(a) ≈ P_n(-1)$ and $f(b) ≈ P_n(1)$)

The algorithm is implemented as python code below:

In [41]:
def integral(f, x0, x1, n, shift = [-1,1], x = False):
    """Integration using trapezoidal rule."""
    step = np.absolute((x0 - x1) / n)
    a, b = shift[0], shift[1]
    area = f(0.5*(b-a)*(x0) + 0.5*(a+b))

    for i in range(1,n):
        if x:
          area += 2 * f(0.5*(b-a)*(x0 + i*step) + 0.5*(a+b))*(x0 + i*step)
        else:
          area += 2 * f(0.5*(b-a)*(x0 + i*step) + 0.5*(a+b))
    
    area *= (step/2)
    
    return area

def pw_linear_one_d(f, mesh):
  steps = len(mesh)
  A = np.zeros((2, steps))
  for i in range(steps-1):
    A[0,i] = (1/2)*integral(f, -1, 1, 5000, [mesh[i], mesh[i+1]])
    A[1,i] = (3/2)*integral(f, -1, 1, 5000, [mesh[i], mesh[i+1]], True)
  result = []
  for j in range(steps-1):
    result.append(A[0,j] + A[1,j]*-1)
  result.append(A[0,j] + A[1,j]*1)
  return result

# **Results**

We test the algorithm presented in the Methods section below. We show that the resulting function will converge towards the exact function for a mesh of higher resolution. We will test it on an arbitrary function $sin(x)$ within the range $[0,π]$. This is done by generating the values of the projections at the points in the mesh and comparing the approximated value to the exact solution using the numpy library:

In [45]:
def f(x):
  return np.sin(x)

test = [3, 5, 10, 100]

for t in test:
  mesh = np.linspace(0, np.pi, t)

  result = pw_linear_one_d(f, mesh)

  exact = []

  for x in mesh:
    exact.append(f(x))

  avg = 0
  for i in range(len(result)):
    avg += np.absolute(result[i] - exact[i])
  print("Average error for a mesh of " + str(len(mesh)) + " points: " + str(avg/len(result)))


Average error for a mesh of 3 points: 0.12940339179115076
Average error for a mesh of 5 points: 0.02948097288685485
Average error for a mesh of 10 points: 0.005764236359716309
Average error for a mesh of 100 points: 0.0001995398183419503


As can be seen from these results, the algorithm produces an accurate result that converges as the mesh grows larger.

# **Discussion**

As expected, we were able to closely approximate the function in $L^2$ using projections onto the first two Legendre polynomials to create piecewise linear approximations. It should be noted that other basis functions for $L^2$ can be utilized for this purpose, and more terms can easily be added in order to increase the accuracy of the results for any given mesh.